EQE512 MATRIX METHODS IN STRUCTURAL ANALYSIS 

---
<h3 align="center">Week 09 - evelopment of solution algorithm using displacement method </h3> 
<h4 align="center">Dr. Ahmet Anıl Dindar (adindar@gtu.edu.tr)</h3> 
<h4 align="center">2020 Fall  </h4> 

---

# This week : 

1. Academic Programs
    
2. OpenSees
    
3. OpenSeesPy
    
4. Truss

---


## Academic Programs

There are many programs developed by the academics since 1970s;
- SAP
- ETABS
- Drain
- IDARC
- Frame

All ara opensource and with no Graphical User  Interface (GUI)

Source: https://nisee.berkeley.edu/elibrary/software.html

<img src="./figures/14-EQE512-Week10-Academics.PNG">

**Do you remember SAP2000?**

<img src="./figures/14-EQE512-Week10-Sap.PNG">


## OpenSees = "Open System for Earthquake Engineering Simulation"

OpenSees history starts in 1999 in Berkeley, California USA. 

OpenSees (the Open System for Earthquake Engineering Simulation) is a proprietary object-oriented, software framework created at the National Science Foundation-sponsored Pacific Earthquake Engineering (PEER) Center. It allows users to create finite element applications for simulating the response of structural and geotechnical systems subjected to earthquakes. This framework was developed by Frank McKenna and Gregory L. Fenves with significant contributions from Michael H. Scott, Terje Haukaas, Armen Der Kiureghian, Remo M. de Souza, Filip C. Filippou, Silvia Mazzoni, and Boris Jeremic. OpenSees is primarily written in C++ and uses several Fortran numerical libraries for linear equation solving.


**The parts of OpenSees**

<img src="https://opensees.berkeley.edu/wiki/images/uml-96ea6309b23a3cf95648d8be3014e598-4976d61b094e206d46b357904a003eac.png">

**In details**

<img src="https://ars.els-cdn.com/content/image/1-s2.0-S2352711017300584-gr2_lrg.jpg">

**TcL**

_OpenSees was developed in TcL language_

```TCL
model basic -ndm 2 -ndf 3
# nodal coordinates:
node 1 0 0
node 2 504 0
node 3 0 432
node 4 504 432 

# bondary conditions:
fix 1 1 1 1
fix 2 1 1 1
fix 3 0 0 0
fix 4 0 0 0

# nodal masses:
mass 3 5.18 0. 0.
mass 4 5.18 0. 0.

# transformation:
geomTransf Linear 1 

# element connectivity:
element elasticBeamColumn 1 1 3 3600 4227 1080000 1
element elasticBeamColumn 2 2 4 3600 4227 1080000 1
element elasticBeamColumn 3 3 4 5760 4227 4423680 1

# recorders
recorder Node -file Node3.out -time -node 3 -dof 1 2 disp
recorder Element -file Element1.out -time -ele 1 force
```

---

**Why TCL?**

TCL was the language where you can do programming in the code in the end of 1990s. 

``` TCL
set IDloadTag 400;	# for uniformSupport excitation
# Uniform EXCITATION: acceleration input
foreach GMdirection $iGMdirection GMfile $iGMfile GMfact $iGMfact {
	incr IDloadTag;
	set inFile $GMdir/$GMfile.at2
	set outFile $GMdir/$GMfile.g3;	# set variable holding new filename (PEER files have .at2/dt2 extension)
	ReadSMDFile $inFile $outFile dt;	# call procedure to convert the ground-motion file
	set GMfatt [expr $g*$GMfact];	# data in input file is in g Unifts -- ACCELERATION TH
	set AccelSeries "Series -dt $dt -filePath $outFile -factor  $GMfatt";	# time series information
	pattern UniformExcitation  $IDloadTag  $GMdirection -accel  $AccelSeries  ;	# create Unifform excitation
}

set Nsteps [expr int($TmaxAnalysis/$DtAnalysis)];
set ok [analyze $Nsteps $DtAnalysis];	# actually perform analysis; returns ok=0 if analysis was successful
```

---

**How to run OpenSees files?**

<img src = "./figures/14-EQE512-Week10-OpenSees.PNG" width="50%">


---

**Process**


<img src = "https://previews.123rf.com/images/pixelsaway/pixelsaway1608/pixelsaway160800114/63291983-abstract-blank-flowchart-or-mind-map-on-a-napkin-with-a-cup-of-coffee.jpg" width="40%">

---

## OpenSeesPy

Why Tcl?

Why not Python?

A group of people started converting OpenSees TcL to OpenSeesPy in 2018. 

<img src="./figures/14-EQE512-Week10-PythonVsTCL.jpg" width="70%">


Source: https://openseespydoc.readthedocs.io/en/latest/#

---

# Example of modeling a Truss in OpenSeesPy

In [ ]:
import openseespy.opensees as  ops
import matplotlib.pyplot as plt

In [ ]:
def onurFunction( force_lateral , force_vertical):
    ops.wipe() # clear all variables in the domain

    ops.model( "basic" , "-ndm",2 , "-ndf",2)
    
    coords = {1 :[0,0] , 2:[3,0], 3:[1.5  , 4]}

    ops.node( 1 , 0 ,0 )
    ops.node( 2 , 3 , 0 )
    ops.node( 3 , 1.5 , 4 )

    ops.fix( 1 , 1 , 1 )
    ops.fix( 2 , 0 , 1 )

    ops.uniaxialMaterial( "Elastic" , 1 , 200_000_000)

    # https://openseespydoc.readthedocs.io/en/latest/src/trussEle.html
    ops.element( "Truss" , 1 , 1 ,2 , 0.01 , 1) 
    ops.element( "Truss" , 2 , 1 ,3 , 0.01 , 1) 
    ops.element( "Truss" , 3 , 3 ,2 , 0.01 , 1) 

    ops.timeSeries( "Linear" , 1)
    ops.pattern("Plain", 1 ,1 )
    ops.load(3 , force_lateral , force_vertical )

    #Solution methods
    # create SOE
    ops.system("BandSPD")
    # create DOF number
    ops.numberer("RCM")
    # create constraint handler
    ops.constraints("Plain")
    # create integrator
    ops.integrator("LoadControl", 1.0)
    # create algorithm
    ops.algorithm("Linear")
    # create analysis object
    ops.analysis("Static")
    # perform the analysis
    ops.analyze(1)

    coords_def = {}
    scale  = 100
    for node_value in [1,2,3]:
        
        values_1 = ops.nodeDisp( node_value , 1)
        values_2 = ops.nodeDisp( node_value , 2)
        
        coords_def[node_value] = [coords[node_value][0] + scale * values_1  , coords[node_value][1] + scale * values_2 ]
        
        print( f"Node number is = {node_value} , displacement = {round(values_1,6)}m")
        
    # plotting
    plt.figure()
    plt.grid()
    for node_value in [1,2,3]:
        plt.plot( coords[ node_value][0] , coords[ node_value][1] ,"k.")
        plt.plot( coords_def[ node_value][0] , coords_def[ node_value][1] ,"r*", ms = 10)
    plt.plot(coords[1][0] , coords[1][1], "^" , ms=20 )
    plt.plot(coords_def[2][0] , coords_def[2][1], "o" , ms=20 )

    return( coords_def ) 

In [ ]:
coordinate_deformed = onurFunction( 1000 , 0)

---

**NEXT WEEK**

<table >
<tr>
<td>
<img src="http://worshiphousemedia.s3.amazonaws.com/images/main/s/st/bnt/st/seeyounextweek1.jpg" width="40%">
</td>
<td align="left"><strong style="font-size: 20px;">
The topics for the next week 
<br><br>

1- Assignment : #4 assignment
<br><br>

<br>
</td>
</tr>
</table>